In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
import neurokit2 as nk
import matplotlib.pyplot as plt

from read_record import Record,RecordReader,plot_signal_with_annotation
from scanning_window import scan_record
from collections import Counter
from transform_image import get_combined_beat_image

from neurokit2 import ecg_clean

In [2]:
folder_path="D:/CPSC2021/Training_set_II"

In [ ]:
file_name="data_89_13"
reading_record=RecordReader.read(folder_path,file_name,0,0,None)
reading_record._Record__label

In [ ]:
Counter(reading_record._Record__aux)

In [ ]:
reading_record.get_afib_interval()

In [ ]:
reading_record.get_nsr_interval()

In [ ]:
reading_record.plot_signal_with_annotation()

In [ ]:
test_scan=scan_record(reading_record,window_width=30)
test_scan

In [ ]:
Counter(test_scan.true_class)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(list(              ))

In [ ]:
signal=np.array(list(test_scan.iloc[431][0:5999]))
clean_signal=nk.ecg_clean(signal,200)
plt.figure(figsize=(15,6))
plt.plot(clean_signal)


In [ ]:
plot_signal_with_annotation(clean_signal,
                            test_scan.iloc[431]['beat_annotation_symbols'],
                             test_scan.iloc[431]['annotated_samples'],200,
                             'r.', figsize=(15, 6))

In [ ]:
# # Filter the DataFrame for rows where 'true_class' is 'pac'
# pac_df = test_scan[test_scan['true_class'] == 'PAC']

# # Save the filtered DataFrame to a CSV file
# pac_df.to_csv('data_104_21_pac.csv', index=False)

In [ ]:
# # Select rows from 30 to 60 (note that the end index is exclusive)
# selected_rows = test_scan.iloc[ 6:775]
# # Save the selected rows to a CSV file
# selected_rows.to_csv('data_89_13_pac.csv', index=False)

In [ ]:
get_combined_beat_image(signal=clean_signal,
                        bpm=test_scan.iloc[431]['avg_heart_rate'],
                        voltage_range=[-3, 3], 
                        folder_name='test', img_name='Test_beat')

In [3]:
record_files=[file[:-4] for file in os.listdir(folder_path) if file.endswith('atr')]
#record_files

In [14]:
def process_all_records(folder_path, type=None):
    """
    Process all ECG records in a specified folder, segment them, filter based on a specified type,
    and generate corresponding CSV files and combined beat images.

    Parameters:
    - folder_path (str): The path to the folder containing the ECG record files.
    - type (str): The type of segments to filter (e.g., 'Pure_NSR'). If None, no filtering is applied.

    Returns:
    None

    Raises:
    ValueError: If the specified folder_path does not exist or if an invalid type is provided.
    """
    
    # Create the CSV folder if it doesn't exist
    csv_folder = f'{type}_data'
    if not os.path.exists(csv_folder):
        os.makedirs(csv_folder)

    # Loop through each file in record_files
    for file in record_files:
        print(f'{file}')
        csv_file_path = os.path.join(csv_folder, f"{file}_{type}.csv")
        
        # Check if the CSV file already exists
        if os.path.exists(csv_file_path):
            print(f"{file}_{type}.csv file is already created in {csv_folder}")
        else:        
            # Read the ECG record
            record = RecordReader.read(folder_path, file, 0, 0, None)
            if record._Record__label == 'non atrial fibrillation':
                # Segment the ECG record
                segments = scan_record(record, window_width=30)
                # Filter segments based on the specified type (e.g., 'Pure_NSR')
                buffer_dataframe = segments[segments['true_class'] == f'{type}']
                
                # Save filtered data to CSV
                buffer_dataframe.to_csv(csv_file_path, index=False)
                
                #print(f"CSV file created: {csv_file_path}")

                # Process each segment
                for i in range(len(buffer_dataframe)):
                    signal = np.array(list(buffer_dataframe.iloc[i][0:5999]))
                    clean_signal = nk.ecg_clean(signal, 200)
                    # Generate a combined beat image
                    get_combined_beat_image(signal=clean_signal,
                                            bpm=buffer_dataframe.iloc[i]['avg_heart_rate'],
                                            voltage_range=[-3, 3],
                                            folder_name=f'{type}_image', img_name=f'{file}_{i}')



In [15]:
process_all_records(folder_path,'Pure_NSR')


data_100_1
data_100_1_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_10
data_100_10_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_11
data_100_11_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_12
data_100_12_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_13
data_100_13_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_2
data_100_2_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_3
data_100_3_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_4
data_100_4_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_5
data_100_5_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_6
data_100_6_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_7
data_100_7_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_8
data_100_8_Pure_NSR.csv file is already created in Pure_NSR_data
data_100_9
data_100_9_Pure_NSR.csv file is already created in Pure_NSR_data
data